In [ ]:
% cd /content/drive/MyDrive/anotate/OBJ_dataset

In [ ]:
! wget "https://www.dmu-net.org/zip_files/DMUNet_OBJ_format.zip"

In [ ]:
 !unzip dmu.zip

In [ ]:
import os
import numpy as np
import skimage.io as io
import skimage.transform as transform
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline
io.use_plugin('pil')
from PIL import Image
import os
import random

raw_image_path="/content/drive/MyDrive/anotate/OBJ_dataset/Actuator"
backgroud_folder_path="/content/drive/MyDrive/anotate/OBJ_dataset/background"
saved_path="/content/drive/MyDrive/anotate/OBJ_dataset/Actuator_Back"
synthatic_image_name="ACT"
saved_folder_name="Actuator_Back"
!mkdir "/content/drive/MyDrive/anotate/OBJ_dataset/Actuator_Back"


for i in filee:
  I = io.imread(i)/255.0
  # Cut out the foreground layer
  foreground = I.copy()
  foreground[foreground>=0.9]=0 # Setting surrounding pixels to zero



  ### Apply augmentations on the foreground.

  def foregroundAug(foreground):
      # Random rotation, zoom, translation
      angle = np.random.randint(-10,10)*(np.pi/180.0) # Convert to radians
      zoom = np.random.random()*0.4 + 0.8 # Zoom in range [0.8,1.2)
      t_x = np.random.randint(0, int(foreground.shape[1]/3))
      t_y = np.random.randint(0, int(foreground.shape[0]/3))

      tform = transform.AffineTransform(scale=(0.6,0.6),
                                  rotation=angle,
                                  translation=(t_x, t_y))
      foreground = transform.warp(foreground, tform.inverse)

      # Random horizontal flip with 0.5 probability
      if(np.random.randint(0,100)>=50):
          foreground = foreground[:, ::-1]
          
      return foreground

  foreground_new = foregroundAug(foreground)
  io.imsave("fname.jpg", foreground_new)



  # Create a mask for this new foreground object
  def getForegroundMask(foreground):
      mask_new = foreground.copy()[:,:,0]
      mask_new[mask_new>0] = 1
      return mask_new

  mask_new = getForegroundMask(foreground_new)
  io.imsave("mask_new.jpg", mask_new)
  #plt.imshow(mask_new)
  #plt.axis('on')
  #plt.show()


  

  background_list=list(())
  for subdir, dirs, files in os.walk(backgroud_folder_path):
      for file in files:
          #print os.path.join(subdir, file)
          filepath = subdir + os.sep + file

          if filepath.endswith(".jpg"):
              background_list.append(filepath)

  for i in background_list:
      nam=(random.randint(0,11000))
      background = Image.open(i)

      foreground = Image.open('fname.jpg')
      mask_im=Image.open('mask_new.jpg')
      #back_im = im1.copy()
      background.paste(foreground, (0, 0),mask_im)

      background.save((saved_path+"/"+ synthatic_image_name + str(nam) +'.jpg'), quality=95)

      nz = np.nonzero(mask_im)
      bbox = [np.min(nz[0]), np.min(nz[1]), np.max(nz[0]), np.max(nz[1])]

      width, height = background.size

      import xml.etree.cElementTree as ET

      root = ET.Element('annotation')

      ET.SubElement(root, 'folder').text = saved_folder_name # set correct folder name
      ET.SubElement(root, 'filename').text = synthatic_image_name + str(nam) +'.jpg'
      ET.SubElement(root, 'path').text = saved_path +synthatic_image_name + str(nam) +'.jpg'


      size = ET.SubElement(root, 'size')
      ET.SubElement(size, 'width').text = str(width)
      ET.SubElement(size, 'height').text = str(height)
      ET.SubElement(size, 'depth').text = str(3)

      ET.SubElement(root, 'segmented').text = '0'

      #for box in rectangles:
      name = "actuator"# class name

      xmin = bbox[1] #set correct index
      ymin = bbox[0] #set correct index
      xmax = bbox[3]  #set correct index
      ymax = bbox[2] #set correct index

      obj = ET.SubElement(root, 'object')
      ET.SubElement(obj, 'name').text = name
      ET.SubElement(obj, 'pose').text = 'Unspecified'
      ET.SubElement(obj, 'truncated').text = '0'
      ET.SubElement(obj, 'occluded').text = '0'
      ET.SubElement(obj, 'difficult').text = '0'

      bx = ET.SubElement(obj, 'bndbox')
      ET.SubElement(bx, 'xmin').text = str(xmin)
      ET.SubElement(bx, 'ymin').text = str(ymin)
      ET.SubElement(bx, 'xmax').text = str(xmax)
      ET.SubElement(bx, 'ymax').text = str(ymax)


      tree = ET.ElementTree(root)
      tree.write((saved_path+"/" +synthatic_image_name + str(nam) +'.xml'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: fname.jpg is a low contrast image
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: mask_new.jpg is a low contrast image
